In [1]:
# Import dependencies
import pandas as pd
pd.set_option('display.max_columns', 500)

import plotly.graph_objects as go
import chart_studio.plotly as py

## Data Processing

In [2]:
# Read in Yankees schedule from FanGraphs
sched_df = pd.read_csv('Resources/Data/yankees_schedule.csv')
sched_df.head()

,Gm#,Date,Unnamed: 2,Tm,Unnamed: 4,Opp,W/L,W/L/wo,R,RA,Inn,W-L,Rank,GB,Win,Loss,Save,Time,D/N,Attendance,cLI,Streak,Orig. Scheduled
0,1,"Friday, Apr 8",boxscore,NYY,NaN,BOS,W,W-wo,6,5,11.0,Jan-00,1,Tied,King,Crawford,NaN,3:56,D,"46,097",0.97,+,NaN
1,2,"Saturday, Apr 9",boxscore,NYY,NaN,BOS,W,W,4,2,NaN,Feb-00,1,Tied,Luetge,Pivetta,Chapman,2:58,D,"46,882",1.03,++,NaN
2,3,"Sunday, Apr 10",boxscore,NYY,NaN,BOS,L,L,3,4,NaN,1-Feb,2,1,Crawford,Schmidt,Diekman,3:40,N,"40,108",1.04,-,NaN
3,4,"Monday, Apr 11",boxscore,NYY,NaN,TOR,L,L,0,3,NaN,2-Feb,3,1,Manoah,Taillon,Romano,3:03,N,"26,211",1.05,--,NaN
4,5,"Tuesday, Apr 12",boxscore,NYY,NaN,TOR,W,W,4,0,NaN,2-Mar,2,1,Holmes,Kikuchi,NaN,3:07,N,"25,068",1.03,+,NaN


In [3]:
# Remove unnecessary columns
sched_df = sched_df[['Gm#', 'Date', 'Tm', 'Opp', 'W/L', 'R', 'RA', 'GB']]
sched_df.head()

,Gm#,Date,Tm,Opp,W/L,R,RA,GB
0,1,"Friday, Apr 8",NYY,BOS,W,6,5,Tied
1,2,"Saturday, Apr 9",NYY,BOS,W,4,2,Tied
2,3,"Sunday, Apr 10",NYY,BOS,L,3,4,1
3,4,"Monday, Apr 11",NYY,TOR,L,0,3,1
4,5,"Tuesday, Apr 12",NYY,TOR,W,4,0,1


In [4]:
# Remove unnecessary rows
#sched_df = sched_df.loc[(sched_df['Gm#'] >= 5) & (sched_df['Gm#'] <= 133)]

In [5]:
sched_df['Win T/F'] = sched_df['W/L'].apply(lambda x: 1 if (x=="W") else 0)
sched_df['Wins'] = sched_df['Win T/F'].cumsum()
sched_df['Win_Pace'] = sched_df['Wins'] / sched_df['Gm#'] * 162
sched_df.head()

,Gm#,Date,Tm,Opp,W/L,R,RA,GB,Win T/F,Wins,Win_Pace
0,1,"Friday, Apr 8",NYY,BOS,W,6,5,Tied,1,1,162.0
1,2,"Saturday, Apr 9",NYY,BOS,W,4,2,Tied,1,2,162.0
2,3,"Sunday, Apr 10",NYY,BOS,L,3,4,1,0,2,108.0
3,4,"Monday, Apr 11",NYY,TOR,L,0,3,1,0,2,81.0
4,5,"Tuesday, Apr 12",NYY,TOR,W,4,0,1,1,3,97.2


In [6]:
# Convert Date column to datetime
sched_df['Date'] = sched_df['Date'] + ' 2022'
sched_df['Date'] = pd.to_datetime(sched_df['Date'], format="%A, %b %d %Y")
sched_df.head()

,Gm#,Date,Tm,Opp,W/L,R,RA,GB,Win T/F,Wins,Win_Pace
0,1,2022-04-08,NYY,BOS,W,6,5,Tied,1,1,162.0
1,2,2022-04-09,NYY,BOS,W,4,2,Tied,1,2,162.0
2,3,2022-04-10,NYY,BOS,L,3,4,1,0,2,108.0
3,4,2022-04-11,NYY,TOR,L,0,3,1,0,2,81.0
4,5,2022-04-12,NYY,TOR,W,4,0,1,1,3,97.2


## Viz

In [7]:
# Create variables for use in chart

# data variables
x = sched_df['Gm#']
y1 = sched_df['Wins']
#text1 = [f'<b>{y}</b>' for y in sched_df['Wins']]
y2 = sched_df['Win_Pace']
#text2 = [f'<b>{y}</b>' for y in scherzer_df['SO']]


# # hover label variables
# teams = lemahieu_df['Team']
# OBPs = lemahieu_df['OBP']
# fWARs = lemahieu_df['fWAR']
# wRCs = lemahieu_df['wRC+']

# hover = [
#     f'       {tm}<br><b>OBP:    {obp}th<br>fWAR:  {fwar}th<br>wRC+:  {wrc}th</b>'
#     for tm, obp, fwar, wrc in
#     zip(teams, OBPs, fWARs, wRCs)
# ]


# color variables
navy = '#0C2340'
gray = '#C4CED3'

In [10]:
# Create traces
fig = go.Figure()


fig.add_trace(
    go.Scatter(
        x=x, y=y2, mode='lines', name='<b>Win Pace</b>', line_color='#a2f5c1', #text=text2
        #customdata=hover, hovertemplate='%{customdata}<extra></extra>'
))

fig.add_trace(
    go.Scatter(
        x=x, y=y1, mode='lines', name='<b>Wins</b>', line_color='#20b358', #text=text1
        #customdata=hover, hovertemplate='%{customdata}<extra></extra>'
))


# Add traces for annotations and text for end of lines
for i, d in enumerate(fig.data):
    fig.add_trace(go.Scatter(x=[d.x[64]],
                             y=[d.y[64]],
                             mode='markers+text',
                             text=['<b>' + str(int(round(y))) + '</b>' for y in [d.y[64]]],
                             textfont = dict(
                                 size=16,
                                 color=navy),
                             textposition='middle center',
                             marker = dict(
                                 size=40,
                                 color=d.line.color,
                                 line=dict(
                                     width=3,
                                     color=navy)),
                             hoverinfo='skip',
                             showlegend=False))
    
    fig.add_trace(go.Scatter(x=[d.x[-1]],
                             y=[d.y[-1]],
                             mode='markers+text',
                             text=['<b>' + str(int(round(y))) + '</b>' for y in [d.y[-1]]],
                             textfont = dict(
                                 size=16,
                                 color=navy),
                             textposition='middle center',
                             marker = dict(
                                 size=40,
                                 color = d.line.color,
                                 line=dict(
                                     width=3,
                                     color=navy)),
                             hoverinfo='skip',
                             showlegend=False))

# Add traces for annotations and text for end of lines
# for i, d in enumerate(fig.data):
#     fig.add_trace(go.Scatter(x=[d.x[-1]],
#                              y=[d.y[-1]],
#                              mode='markers+text',
#                              text=['<b>' + str(int(round(y))) + '</b>' for y in [d.y[-1]]],
#                              textfont = dict(
#                                  size=16,
#                                  color=navy),
#                              textposition='middle center',
#                              marker = dict(
#                                  size=40,
#                                  color = d.line.color,
#                                  line=dict(
#                                      width=3,
#                                      color=navy)),
#                              hoverinfo='skip',
#                              showlegend=False))
    

# Format traces
fig.update_traces(
#     textposition=['middle center'] + ['middle center']*17,
#     textfont=dict(
#         size=18,
#         color='white'),
#     marker=dict(
#         size=25,
#         color=d.line.color),
    line=dict(
        shape='spline',
        smoothing=1.3,
        width=20)
)


# Format axes
fig.update_xaxes(
#     dtick='M1',
#     tickformat='<b>%b</b>',
#     ticklabelmode='period',
    title=dict(
        text="<b>Games Played</b>",
        font=dict(
            size=26,
            color='white')),
    tickmode='array',
    ticktext=['<b>' + x + '</b>' for x in ['0','20','40','60','80','100','120']],
    tickvals=[0, 20, 40, 60, 80, 100, 120],
    tickfont=dict(
        size=24,
        color=gray),
    gridcolor='rgb(20,45,75)',
    gridwidth=2,
    zeroline=False
)

fig.update_yaxes(
    tickmode='array',
    ticktext=['<b>' + x + '</b>' for x in ['0','25','50','75','100','125','150','175']],
    tickvals=[0, 25, 50, 75, 100, 125, 150, 175],
    tickfont=dict(
        size=24,
        color=gray),
    zeroline=False,
    zerolinecolor='rgb(20,45,75)',
    zerolinewidth=2,
    gridcolor='rgb(20,45,75)',
    gridwidth=2
)


# Format layout
fig.update_layout(
    title=dict(
        text="<b>History to Mystery</b><br>The Collapse of a Record-Setting Win Pace",
        y=0.9,
        x=0.5,
        xanchor='center',
        yanchor='top',
        font=dict(
            size=30,
            color='white')),
    xaxis_range=[-0.2,137],
    yaxis_range=[-5,160],
    legend=dict(
        title_text='',
        orientation='h',
        traceorder='reversed',
        yanchor="top",
        y=1.01,
        xanchor="left",
        x=0.357,
        font=dict(
            size=18,
            color=navy),
        bordercolor=gray,
        borderwidth=2,
        bgcolor='white'),
    hoverlabel=dict(
        bgcolor=navy,
        font_size=16),
    paper_bgcolor=navy,
    plot_bgcolor=navy,
    margin=dict(
        t=120,
        #b=70,
        pad=3)
)


# Save chart online to Plotly Chart Studio
#py.iplot(fig, filename='DJ-LeMahieu')